# Prepare Mask Data by moriitkys (2021/02)
LICENSE : MIT

<img src="Documents/imgs/AugDataSet_structure_all.png" width="400">

# 1, Move Mask Data 
## move c_raw_white_mask into "0000x_raw_mask_prep" folder
<img src="Documents/imgs/prepare_mask_data_0001.PNG" width="200">

In [4]:
# ----- Move c_raw_white_mask into "0000x_raw_mask_prep" folder -----

import shutil
import glob
import os
import csv

import numpy as np
import random
import matplotlib.pyplot as plt

import PIL
from PIL import Image
import cv2

from pathlib import Path
import shutil

from sklearn.model_selection import train_test_split

import mylib.create_panel_for_selectfolder as create_panel
import mylib.config as config
config_param = config.ConfigParameters()

# ------ Setting panels ------
import tkinter
from tkinter import messagebox

key_for_searching_foldername = config_param.key_pmd_mmd
setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
setting_panel.create_buttons()#If you push "start", exit this line.

list_selected_folders = setting_panel.list_selected_folders
print("list_selected_folders", list_selected_folders)
#print(setting_panel.dict_object_class_number)

for selected_folder in list_selected_folders:
    output_folder = selected_folder + "_mask_prep"
    input_files = glob.glob(selected_folder + "/*" + key_for_searching_foldername)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    for input_file in input_files:
        output_file = input_file.replace("whitemask", "mask")
        output_file = output_file.replace(selected_folder, selected_folder + "_mask_prep")
        
        shutil.copyfile(input_file, output_file)
        
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data.csv','w')
# https://work-life-enj.com/pyhon-write-csv/
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)
#for i in list_selected_folders:
#    w.writerow(i)
f.close()

print("----- Complete Move Mask Data -----")

list_selected_folders []
----- Complete Move Mask Data -----


# 2, Move Manual-Colored Data
**Move** img0000x_c_raw_mask.png  
 **from** 000x_raw_mask_prep  
 **to** 000x_raw

# 2-1, Manual coloring 手作業での色塗り
- Mask RCNNまたはセマンティックセグメンテーション用にオブジェクトの色塗りをする。以下の画像を参考とする。
- 画像中にただ一つのクラスの一つのオブジェクトのみ存在する場合、<font color="Red">最後の「おまけ」が実行できる。</font>
- 例えばMask RCNNなら、一つのクラスのオブジェクトでそれぞれのオブジェクトを区別するのでそれぞれ段階的に色の塗り分けが必要である。
- セマンティックセグメンテーションであれば同じクラスのオブジェクトは同じ色で塗ってよい。ただし、クラスの色を定義する必要がある。
- Mask RCNNの場合、一枚の画像に違うクラスのオブジェクトが映り込んだ場合はセマンティックセグメンテーションとは違うので注意。  

- Mask RCNN or Semantic Segmentation to color the object. Refer to the image below.
- If there is only one object of only one class in the image, the last <font color="Red">"bonus" can be executed. </font>
- For example, in Mask RCNN, each object is distinguished by one class of objects, so it is necessary to color each object step by step.
- Objects of the same class may be painted in the same color for semantic segmentation. However, you need to define the color of the class.
- Note that in the case of Mask RCNN, if an object of a different class is reflected in one image, it is different from semantic segmentation.
<img src="Documents/imgs/MaskImageDifferenceMRCNN_SS.PNG" width="400">
<img src="Documents/imgs/mrcnn_grayscale.PNG" width="300">

# 2-2, Move Manual-Colored Data
<img src="Documents/imgs/prepare_mask_data_0002.PNG" width="200">

In [5]:
# Move img0000x_c_raw_mask.png from 000x_raw_mask_prep to 000x_raw
import glob
import tkinter
from tkinter import messagebox
import csv
import mylib.create_panel_for_selectfolder as create_panel
import shutil
import mylib.config as config
config_param = config.ConfigParameters()
f = open('./log/prepare_mask_data.csv', 'r')

reader = csv.reader(f)
#header = next(reader)
#list_selected_folders = []
dict_selected_folders = {}
for row in reader:
    #list_selected_folders.append(row[0])
    row[0] = row[0].replace("\\", "/")
    rslash_1st = row[0].rfind('/')
    rslash_2nd = row[0][:rslash_1st].rfind('/')
    object_class_number_name = row[0][rslash_2nd+1 :]
    dict_selected_folders[object_class_number_name] = row[0]
str_selected_folders = "\n".join(dict_selected_folders)
#print(str_selected_folders)

root1 = tkinter.Tk()
# You can't use root1 = tkinter.withdraw()

# ----- Yes・No Confirm Target Dir -----
text_message_window = "現在のマスク作成対象フォルダは以下が選択されています。\n"+\
                        "The following is selected as the current mask creation target folder.\n"+\
                        "処理対象フォルダを変更しますか。\n"+\
                        "Do you want to change the processing target folder?\n"+\
                        str_selected_folders
                        
ret = messagebox.askyesno('Confirm Target Dir', text_message_window)

root1.destroy()


# ----- [Main Process] Move Manual-Colored Data by using list_selected_folders -----
list_selected_folders = []
key_for_searching_foldername = config_param.key_pmd_mmcd
if ret:
    # ------ Setting panels ------
    setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
    setting_panel.create_buttons()#If you push "start", exit this line.

    list_selected_folders = setting_panel.list_selected_folders
    print("list_selected_folders", "\n".join(list_selected_folders))
else:
    for key in dict_selected_folders:
        list_selected_folders.append(dict_selected_folders[key])
        print("list_selected_folders", "\n".join(list_selected_folders))
#target_folder = "./DataSource/rgbdt"
#list_object_class = glob.glob(target_folder + "/*")
#for object_class in list_object_class:
#    list_object_class_number = glob.glob(object_class + "/*")
#    for object_class_number in list_object_class_number:
for selected_folder in list_selected_folders:
    selected_folder_mask_prep = selected_folder + key_for_searching_foldername
    list_mask_prep_imgs = glob.glob(selected_folder_mask_prep + "/*")
    if list_mask_prep_imgs is not None:
        for mask_prep_img in list_mask_prep_imgs:
            shutil.move(mask_prep_img, selected_folder_mask_prep.replace("_mask_prep", ""))
            
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data.csv','w')
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)

f.close()

print("----- Complete Move Manual-Colored Data -----")

----- Complete Move Manual-Colored Data -----


# 3, Crop & Move data 
### Crop the data (RGB & D) according to the thermal data and move the data to the source folder for data expansion.
### データ(RGB & D)をサーマルデータに合わせて切り取り、データ拡張用ソースフォルダへデータ移動する
<img src="Documents/imgs/prepare_mask_data_0003.PNG" width="250">

In [3]:
import os
import glob
import numpy as np
import cv2
import tkinter
from tkinter import messagebox
import csv
import mylib.create_panel_for_selectfolder as create_panel
import mylib.config as config
config_param = config.ConfigParameters()

f = open('./log/prepare_mask_data.csv', 'r')

reader = csv.reader(f)

dict_selected_folders = {}
for row in reader:
    #list_selected_folders.append(row[0])
    row[0] = row[0].replace("\\", "/")
    rslash_1st = row[0].rfind('/')
    rslash_2nd = row[0][:rslash_1st].rfind('/')
    object_class_number_name = row[0][rslash_2nd+1 :]
    dict_selected_folders[object_class_number_name] = row[0]
str_selected_folders = "\n".join(dict_selected_folders)
#print(str_selected_folders)

root1 = tkinter.Tk()

# ----- Yes・No Confirm Target Dir -----
text_message_window = "現在のクロップ&移動対象フォルダは以下が選択されています。\n"+\
                        "The following is selected as the current crop&move target folder.\n"+\
                        "処理対象フォルダを変更しますか。\n"+\
                        "Do you want to change the processing target folder?\n"+\
                        str_selected_folders
                        
ret = messagebox.askyesno('Confirm Target Dir', text_message_window)
root1.destroy()

# ----- [Main Process] -----
list_selected_folders = []
key_for_searching_foldername = config_param.key_pmd_cmd
if ret:
    # ------ Setting panels ------
    setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
    setting_panel.create_buttons()#If you push "start", exit this line.

    list_selected_folders = setting_panel.list_selected_folders
    print("list_selected_folders", list_selected_folders)
else:
    for key in dict_selected_folders:
        list_selected_folders.append(dict_selected_folders[key])

list_crop_target_extention = config_param.list_crop_target_extention
# If the data type is RGBD there is no need to crop
crop_area = config_param.crop_area# xmin, ymin, xmax, ymax

for selected_folder in list_selected_folders:# Like "/0001_raw", "/0001_raw_mask_prep"
    selected_folder = selected_folder
    list_raw_files = glob.glob(selected_folder + "/*")
    if not os.path.exists(selected_folder.replace("_raw", "")):
        os.mkdir(selected_folder.replace("_raw", ""))
    for raw_file in list_raw_files:
        for extention in list_crop_target_extention:
            if extention in raw_file:
                path_output = raw_file
                if "_t_raw.png" in raw_file:
                    path_output = path_output.replace("_raw", "")
                    img = cv2.imread(raw_file)
                    img_out = img
                    #cv2.imwrite(path_output, img_out )
                elif "_t_raw.csv" in raw_file:
                    thermal_csv = np.loadtxt(raw_file, 
                                        delimiter=",",
                                        skiprows=0,
                                        )
                    h, w = thermal_csv.shape[:2]
                    thermal_array = thermal_csv.copy()

                    out_thermal_array = thermal_array
                    # Save csv file
                    #np.savetxt(raw_file.replace("_raw", ""), out_thermal_array, delimiter=',')
                    
                elif "_d_raw.csv" in raw_file:
                    depth_csv = np.loadtxt(raw_file, 
                                        delimiter=",",
                                        skiprows=0,
                                        )
                    h, w = depth_csv.shape[:2]
                    depth_array = depth_csv.copy()

                    out_depth_array = depth_array[crop_area[0][1]:crop_area[1][1], crop_area[0][0]:crop_area[1][0]]
                    # Save csv file
                    np.savetxt(raw_file.replace("_raw", ""), out_depth_array, delimiter=',')
                    
                else:    
                    #if "_c_raw" in raw_file:
                        #path_output = raw_file.replace("_c_raw", "")
                    path_output = path_output.replace("_raw", "")
                    #print(raw_img)
                    #print(path_output)
                    img = cv2.imread(raw_file)
                    # Crop image to match the shape of the thermal data
                    img_out = img[crop_area[0][1]:crop_area[1][1], crop_area[0][0]:crop_area[1][0]]
                    cv2.imwrite(path_output, img_out )
                    #shutil.move(raw_img, path_output)
                
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data.csv','w')
# https://work-life-enj.com/pyhon-write-csv/
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)
#for i in list_selected_folders:
#    w.writerow(i)
f.close()
                
print("----- Complete Crop & Move data -----")

----- Complete Crop & Move data -----


# おまけ
画像中に一つのクラスのオブジェクトのみ写っている＆そのクラスのオブジェクトがただ一つのみ写っている場合  
RGB=(255,255,255) -> (1,1,1)にして_raw_mask_prepから_rawへ移動
<img src="Documents/imgs/prepare_mask_data_0002.PNG" width="200">

In [6]:
# Move img0000x_c_raw_mask.png from 000x_raw_mask_prep to 000x_raw
import glob
import os
import numpy as np
import cv2
import tkinter
from tkinter import messagebox
import csv
import mylib.create_panel_for_selectfolder as create_panel
import mylib.config as config
config_param = config.ConfigParameters()
import shutil

f = open('./log/prepare_mask_data.csv', 'r')

reader = csv.reader(f)
#header = next(reader)
#list_selected_folders = []
dict_selected_folders = {}
for row in reader:
    #list_selected_folders.append(row[0])
    row[0] = row[0].replace("\\", "/")
    rslash_1st = row[0].rfind('/')
    rslash_2nd = row[0][:rslash_1st].rfind('/')
    object_class_number_name = row[0][rslash_2nd+1 :]
    dict_selected_folders[object_class_number_name] = row[0]
str_selected_folders = "\n".join(dict_selected_folders)
#print(str_selected_folders)

root1 = tkinter.Tk()
# You can't use root1 = tkinter.withdraw()

# ----- Yes・No Confirm Target Dir -----
text_message_window = "現在のマスク作成対象フォルダは以下が選択されています。\n"+\
                        "The following is selected as the current mask creation target folder.\n"+\
                        "処理対象フォルダを変更しますか。\n"+\
                        "Do you want to change the processing target folder?\n"+\
                        str_selected_folders
                        
ret = messagebox.askyesno('Confirm Target Dir', text_message_window)

root1.destroy()


# ----- [Main Process] Move Manual-Colored Data by using list_selected_folders -----
list_selected_folders = []
key_for_searching_foldername = config_param.key_pmd_mmcd
if ret:
    # ------ Setting panels ------
    setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
    setting_panel.create_buttons()#If you push "start", exit this line.

    list_selected_folders = setting_panel.list_selected_folders
    print("list_selected_folders", "\n".join(list_selected_folders))
else:
    for key in dict_selected_folders:
        list_selected_folders.append(dict_selected_folders[key])
        print("list_selected_folders", "\n".join(list_selected_folders))
#target_folder = "./DataSource/rgbdt"
#list_object_class = glob.glob(target_folder + "/*")
#for object_class in list_object_class:
#    list_object_class_number = glob.glob(object_class + "/*")
#    for object_class_number in list_object_class_number:
for selected_folder in list_selected_folders:
    selected_folder_mask_prep = selected_folder + "_mask_prep"
    list_mask_prep_imgs = glob.glob(selected_folder_mask_prep + "/*")
    if list_mask_prep_imgs is not None:
        for mask_prep_img in list_mask_prep_imgs:
            src = cv2.imread(mask_prep_img)
            out = src.copy()
            #height, width = src.shape[:2]

            white = [255,255,255]
            black = [0,0,0]
            color_111 = [1,1,1]
            out[np.where((out == white).all(axis=2))] = color_111
            out[np.where((out != color_111).all(axis=2))] = black

            output_file = mask_prep_img.replace("_mask_prep", "")
            cv2.imwrite(output_file, out)
            os.remove(mask_prep_img)
            #shutil.move(mask_prep_img, selected_folder_mask_prep.replace("_mask_prep", ""))
            
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data.csv','w')
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)

f.close()

print("----- Complete Move Manual-Colored Data -----")

----- Complete Move Manual-Colored Data -----
